這份 Notebook 示範 OpenAI Function Calling 的 Extract metadata 用法


In [2]:
# from google.colab import userdata
# openai_api_key = userdata.get('openai_api_key')

In [3]:
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv('OPENAI_API_KEY')

In [4]:
import requests
import json
from pprint import pp

In [6]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=1000, tools=None, tool_choice=None):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  if tools:
    payload["tools"] = tools
  if tool_choice:
    payload["tool_choice"] = tool_choice

  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)

  if response.status_code == 200 :
    return obj["choices"][0]["message"] # 改成回傳上一層 message 物件
  else :
    return obj["error"]

## Fake function 用法: 擷取 metadata

透過一個 fake function，目的是拿結構化的 function 參數

In [7]:
content = """大家下午好，歡迎參加台積電2023年第四季財報發表會及電話會議。很高興再次見到大家。我是台積電投資者關係總監 Jeff Su，也是今天的主持人。今天的活動透過台積電網站 www.tsmc.com 進行網路直播，您也可以在其中下載財報發布資料。
今天活動的形式如下：首先，台積電副總裁兼財務長黃文德爾先生將總結我們2023年第四季和2023年全年的營運情況，然後是我們對2024年第一季的指導。黃，台積電首席執行官，魏長城博士；台積電董事長劉馬克博士將共同發表公司的關鍵資訊。然後台積電董事長劉馬克博士將主持問答環節，我們的三位高階主管將回答您的問題。"""


In [8]:
messages = [
  {"role": "system", "content": "You're a metadata extractor." },
  {"role": "user", "content": content }
]

tools = [
    {
        "type": "function",
        "function": {
            "name": "information_extraction",
            "description": "Extracts the relevant information from the passage.",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "公司名稱",
                    },
                    "season": {
                        "type": "string",
                        "description": "報告季度"
                    },
                    "speakers": {
                        "type": "string",
                        "description": "報告人"
                    },
                }
            },
        },
    }
]

# 這個 tool_choice 參數可以要求 GPT 一定要執行某個函數，預設是 auto 讓 GPT 自行判斷
tool_choice =  {"type": "function", "function": {"name": "information_extraction"}}

response = get_completion(messages, tools=tools, tool_choice=tool_choice)
pp(response)

{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_JHA8qxGArUCxw9dIDSaQxFe7',
                 'type': 'function',
                 'function': {'name': 'information_extraction',
                              'arguments': '{"company_name":"台積電","season":"2023年第四季","speakers":"Jeff '
                                           'Su"}'}}],
 'refusal': None,
 'annotations': []}


In [ ]:
metadata = json.loads(response["tool_calls"][0]["function"]["arguments"])
pp(metadata)



{'company_name': '台積電', 'season': '2023年第四季', 'speakers': 'Jeff Su'}


如果是簡單的 JSON 輸出，不用 function calling 只透過 JSON mode (也許搭配一個 few-shot 範例)，就可以做得很好(可能也比較省 tokens 數)。

不過如果要更嚴格拿到符合 schema 的 JSON 格式，透過 function calling 可以做得更好。
而且，function calling 支援的是 JSON schema 格式: https://json-schema.org/
JSON schema 比你想像中的更厲害...

## 示範使用 Pydantic 來產生 JSON schema

https://docs.pydantic.dev/latest/

Pydantic is the most widely used data validation library for Python.

方便我們將 dict 和 json 做對應轉換


In [10]:
from pydantic import BaseModel, Field
from datetime import date

In [11]:
class Report(BaseModel):
  company_name: str = Field(description="公司名稱")
  season: str = Field(description="報告季度")
  speakers: str = Field(description="報告人")

In [13]:
Report.model_json_schema()

{'properties': {'company_name': {'description': '公司名稱',
   'title': 'Company Name',
   'type': 'string'},
  'season': {'description': '報告季度', 'title': 'Season', 'type': 'string'},
  'speakers': {'description': '報告人', 'title': 'Speakers', 'type': 'string'}},
 'required': ['company_name', 'season', 'speakers'],
 'title': 'Report',
 'type': 'object'}

In [14]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "information_extraction",
            "description": "Extracts the relevant information from the passage.",
            "parameters": Report.model_json_schema()
        },
    }
]

# 這個 tool_choice 參數可以要求 GPT 一定要執行某個函數，預設是 auto 讓 GPT 自行判斷
tool_choice =  {"type": "function", "function": {"name": "information_extraction"}}

response = get_completion(messages, tools=tools, tool_choice=tool_choice)
pp(response)

{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_0MSzKzqAKf19ZACvVLWt6hNU',
                 'type': 'function',
                 'function': {'name': 'information_extraction',
                              'arguments': '{"company_name":"台積電","season":"2023年第四季","speakers":"Jeff '
                                           'Su, 黃文德爾, 魏長城博士, 劉馬克博士"}'}}],
 'refusal': None,
 'annotations': []}


In [16]:
response["tool_calls"][0]["function"]["arguments"]

'{"company_name":"台積電","season":"2023年第四季","speakers":"Jeff Su, 黃文德爾, 魏長城博士, 劉馬克博士"}'

In [18]:
report_obj = Report.model_validate_json(response["tool_calls"][0]["function"]["arguments"])

In [22]:
report_obj


Report(company_name='台積電', season='2023年第四季', speakers='Jeff Su, 黃文德爾, 魏長城博士, 劉馬克博士')

## 威力加強版

上述只是簡單的 dict，如果是比較複雜的 json 結構呢?

In [23]:
from typing import List

class Speaker(BaseModel):
  name: str

class ReportV2(BaseModel):
  company_name: str = Field(description="公司名稱")
  season: str = Field(description="報告季度")
  speakers: List[Speaker] = Field(description="報告人")

In [24]:
ReportV2.model_json_schema()

{'$defs': {'Speaker': {'properties': {'name': {'title': 'Name',
     'type': 'string'}},
   'required': ['name'],
   'title': 'Speaker',
   'type': 'object'}},
 'properties': {'company_name': {'description': '公司名稱',
   'title': 'Company Name',
   'type': 'string'},
  'season': {'description': '報告季度', 'title': 'Season', 'type': 'string'},
  'speakers': {'description': '報告人',
   'items': {'$ref': '#/$defs/Speaker'},
   'title': 'Speakers',
   'type': 'array'}},
 'required': ['company_name', 'season', 'speakers'],
 'title': 'ReportV2',
 'type': 'object'}

In [27]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "information_extraction",
            "description": "Extracts the relevant information from the passage.",
            "parameters": ReportV2.model_json_schema()
        },
    }
]

tool_choice =  {"type": "function", "function": {"name": "information_extraction"}}

response = get_completion(messages, tools=tools, tool_choice=tool_choice)
pp(response)



{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_B4b37mgORIErntLhHi4mtzoW',
                 'type': 'function',
                 'function': {'name': 'information_extraction',
                              'arguments': '{"company_name":"台積電","season":"2023年第四季","speakers":[{"name":"Jeff '
                                           'Su"},{"name":"黃文德爾"},{"name":"魏長城博士"},{"name":"劉馬克博士"}]}'}}],
 'refusal': None,
 'annotations': []}


In [28]:
report_obj = ReportV2.model_validate_json(response["tool_calls"][0]["function"]["arguments"])

In [29]:
report_obj

ReportV2(company_name='台積電', season='2023年第四季', speakers=[Speaker(name='Jeff Su'), Speaker(name='黃文德爾'), Speaker(name='魏長城博士'), Speaker(name='劉馬克博士')])

## Classification 也可以用 Function Calling 做

json schema 支援 enum typ

而且也很方便加上 CoT

回傳是結構化的所以很好拆開

In [30]:
# 這是練習用的 dataset，有 title, description 以及標註的正確分類 category
url = "https://ihower.tw/data/books-dataset-200.csv"
response = requests.get(url)

In [32]:
import pandas as pd
from io import StringIO
pd.read_csv(StringIO(response.text))

,title,description,category,tags
0,統計學：重點觀念與題解 (下) 3/e (適用: 大學用書．高普考．商管所),1. 本書涵蓋範圍既深且廣，難度可輕鬆應付頂尖大學各系所。\r\n2. 本書有各種經濟學與財...,機率統計學 Probability-and-statistics,統計學
1,新時代 乙級數位電子術科含學科題庫實作寶典 - 使用 KiCad/Quartus II / ...,1.依勞動部最新公告試題編而成，專爲有意參加「數位電子乙級技術士技能檢定」之人士編寫，並適合...,技能檢定乙級 Skilltest-b,"技能檢定乙級,考試認證"
2,樂高 SPIKE 機器人創意專題實作 -- 使用 LEGO Education SPIKE ...,1.透過「螺旋式教學」提供一套具有邏輯先後順序教材，由具體到抽象，由簡單到複雜的創客教材。\...,機器人製作 Robots,"樂高,機器人"
3,AI 生成時代：從 ChatGPT 到繪圖、音樂、影片，利用智能創作自我加值、簡化工作，成為...,未來2～5年將是AI 生成的時代\r\n史上用戶最快破億、開啟產業新動能的技術，你跟上了嗎？...,人工智慧,ChatGPT
4,都問 AI 吧！ChatGPT 上手的第一本書,"""寫文案、腳本，編教材，做專案，作業輔助\r\n\r\n都可以問AI，但是——\r\n\r\...",ChatGPT,ChatGPT
...,...,...,...,...
194,Autodesk Civil 3D 2024 from Start to Finish: A...,Master Autodesk Civil 3D 2023 to develop real ...,AutoCAD,AutoDESK
195,Network Programming and Automation Essentials:...,Unleash the power of automation by mastering n...,Go 程式語言,"Go,Python"
196,Internet of Things for Smart Buildings: Levera...,Harness the full potential of IoT in your buil...,物聯網 IoT,物聯網IoT
197,AI and Business Rule Engines for Excel Power U...,A power-packed manual to enhance your decision...,Excel,"Excel,人工智慧"


In [35]:
import csv
import random

f = StringIO(response.text)
reader = csv.DictReader(f)
data = list(reader)

# 隨機選擇20筆資料來做實驗就好了
selected_data = random.sample(data, 20)

# 印出選擇的資料
for row in selected_data:
    print(row)

{'title': '線性代數（原書第10版）', 'description': '本書結合大量應用和實例詳細介紹線性代數的基本概念、基本定理與知識點，\r\n主要內容包括：矩陣與方程組、行列式、向量空間、線性變換、正交性、特徵值、數值線性代數和標準型等．\r\n為幫助讀者鞏固所學的基本概念和基本定理，書中每一節後都配有練習題，並在每一章後提供了MATLAB練習題和測試題．\r\n本書敘述簡潔，通俗易懂，理論與應用相結合，\r\n適合作為高等院校本科生“線性代數”課程的教材，同時也可作為工程技術人員的參考書．\r\n', 'category': '線性代數 Linear-algebra', 'tags': '線性代數'}
{'title': '概率與統計:面向計算機專業:原書第3版', 'description': '本書從概率論的基礎開始，帶領學生學習如計算機模擬、蒙特卡羅方法、隨機過程、\r\n馬爾可夫鏈、排隊系統、統計推斷和回歸等廣泛應用於現代計算機科學、計算機工程、軟件工程以及相關領域的重要內容。\r\n第一部分介紹概率和隨機變量。\r\n第二部分講解隨機過程，第三部分引入統計學的基礎知識，附錄部分給出了必要的微積分內容。\r\n另外，R和MATLAB的使用貫穿本書。\r\n', 'category': '機率統計學 Probability-and-statistics', 'tags': '統計學'}
{'title': '概率論與數理統計', 'description': '本書主要內容包括概率論的基本概念、一維隨機變量及其分佈、多維隨機變量及其分佈、隨機變量的數字特徵、大數定律及中心極限定理、數理統計的基本概念、參數估計、假設檢驗和回歸分析。每章附章節思維導圖，數學實驗和軟件求解。\r\n本書適合應用型本科理工類，經管類和其它非數學專業教學用書，也可以作為工程技術人員的參考書。', 'category': '機率統計學 Probability-and-statistics', 'tags': '統計學'}
{'title': 'Go 語言設計模式', 'description': '本書聚焦於Go語言設計模式的知識與應用。\r\n全書共6章，分別為設計模式入門、建型設計模式、結構型設計模式、\r\n行為型設計模式、設計模式擴展、設計模式與軟件架構。\r

In [36]:
df = pd.DataFrame(selected_data)

In [37]:
category_str = "3D 列印,Arduino,C 程式語言,C++ 程式語言,CI/CD,ChatGPT,Computer Vision,Computer-Science,Computer-architecture,Data Science,Data-visualization,DeepLearning,Domain-Driven Design,Engineer self-growth,Go 程式語言,Information-management,Java,Linux,Machine Learning,Operating-system,Python,R 語言,Reinforcement,UI/UX,Version Control,Visual C#,人工智慧,化學 Chemistry,區塊鏈與金融科技,地理資訊系統 Gis,大數據 Big-data,微軟技術,搜尋引擎優化 SEO,數學,機器人製作 Robots,物聯網 IoT,管理與領導 Management-leadership,網站開發,網頁設計,職涯發展,行動通訊 Mobile-communication,行銷/網路行銷 Marketing,資料庫,資訊安全,軟體工程,軟體架構,軟體測試,雲端運算,電力電子 Power-electronics,電機學 Electric-machinery,電路學 Electric-circuits"
category_arr = category_str.split(",")

In [38]:
from typing import Literal
from typing import Iterable

In [40]:
class Category(BaseModel):
    reasoning: str # CoT
    name: Literal[tuple(category_arr)]

In [41]:
Category.model_json_schema()

{'properties': {'reasoning': {'title': 'Reasoning', 'type': 'string'},
  'name': {'enum': ['3D 列印',
    'Arduino',
    'C 程式語言',
    'C++ 程式語言',
    'CI/CD',
    'ChatGPT',
    'Computer Vision',
    'Computer-Science',
    'Computer-architecture',
    'Data Science',
    'Data-visualization',
    'DeepLearning',
    'Domain-Driven Design',
    'Engineer self-growth',
    'Go 程式語言',
    'Information-management',
    'Java',
    'Linux',
    'Machine Learning',
    'Operating-system',
    'Python',
    'R 語言',
    'Reinforcement',
    'UI/UX',
    'Version Control',
    'Visual C#',
    '人工智慧',
    '化學 Chemistry',
    '區塊鏈與金融科技',
    '地理資訊系統 Gis',
    '大數據 Big-data',
    '微軟技術',
    '搜尋引擎優化 SEO',
    '數學',
    '機器人製作 Robots',
    '物聯網 IoT',
    '管理與領導 Management-leadership',
    '網站開發',
    '網頁設計',
    '職涯發展',
    '行動通訊 Mobile-communication',
    '行銷/網路行銷 Marketing',
    '資料庫',
    '資訊安全',
    '軟體工程',
    '軟體架構',
    '軟體測試',
    '雲端運算',
    '電力電子 Power-electronics',
    '電機學 Electric-ma

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "classify_category",
            "description": "Classify category",
            "parameters": Category.model_json_schema()
        },
    }
]

tool_choice =  {"type": "function", "function": {"name": "classify_category"}}



In [47]:
# 隨機選擇 5 筆資料來做實驗就好了
selected_data = random.sample(data, 5)

# 印出選擇的資料
for row in selected_data:

    messages = [
        {"role": "system", "content": "You are a helpful assistant that extracts information from the passage."},
        {"role": "user", "content": f"Extract the information from the passage. <content>{row['title']},{row['description']}</content>"}
    ]

    tool_choice =  {"type": "function", "function": {"name": "classify_category"}}

    response = get_completion(messages, tools=tools, tool_choice=tool_choice)
    pp(response)

{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_2YR4wbgib6L4sWja6fgn6KYY',
                 'type': 'function',
                 'function': {'name': 'classify_category',
                              'arguments': '{"reasoning":"The passage provides '
                                           'information about natural language '
                                           'processing, including basic '
                                           'knowledge and advanced models, as '
                                           'well as practical applications '
                                           'such as pre-trained models, text '
                                           'classification, machine reading '
                                           'comprehension, named entity '
                                           'recognition, text generation, '
                                           'model distillation and pruning, '
                              

Pydantic 還有 Validation 功能

In [44]:
from pydantic import ValidationError

In [48]:
for index, row in df.iterrows():
    messages = [
        {
          "role": "system",
          "content": "請根據以下書籍資訊，選擇最適合的技能分類" },
        {
            "role": "user",
            "content": f"書名: {row['title']} \n 描述: {row['description']}"
        }
    ]
    response = response = get_completion(messages, model="gpt-3.5-turbo", tools=tools, tool_choice=tool_choice)

    try:
      data = Category.model_validate_json(response["tool_calls"][0]["function"]["arguments"])
      df.at[index, 'reasoning'] = data.reasoning
      df.at[index, 'predict'] = data.name
    except ValidationError as e:
      df.at[index, 'predict'] = f"Error: {e}"

df

,title,description,category,tags,reasoning,predict
0,線性代數（原書第10版）,本書結合大量應用和實例詳細介紹線性代數的基本概念、基本定理與知識點，\r\n主要內容包括：矩...,線性代數 Linear-algebra,線性代數,線性代數的基本概念、基本定理、矩陣與方程組、向量空間等內容,數學
1,概率與統計:面向計算機專業:原書第3版,本書從概率論的基礎開始，帶領學生學習如計算機模擬、蒙特卡羅方法、隨機過程、\r\n馬爾可夫鏈...,機率統計學 Probability-and-statistics,統計學,NaN,Error: 1 validation error for Category\nname\n...
2,概率論與數理統計,本書主要內容包括概率論的基本概念、一維隨機變量及其分佈、多維隨機變量及其分佈、隨機變量的數字...,機率統計學 Probability-and-statistics,統計學,概率論與數理統計涵蓋概率論、數理統計基本概念、參數估計、假設檢驗等內容，適合應用型本科理工類...,數學
3,Go 語言設計模式,本書聚焦於Go語言設計模式的知識與應用。\r\n全書共6章，分別為設計模式入門、建型設計模式...,Go 程式語言,Go,NaN,Error: 1 validation error for Category\nname\n...
4,Gender in AI and Robotics: The Gender Challeng...,Why AI does not include gender in its agenda? ...,人工智慧,"人工智慧,機器人",The book discusses the role of gender in AI an...,人工智慧
5,高維統計學非漸近視角,本書對高維統計學進行了詳盡介紹，\r\n重點介紹核心方法論和理論——包括尾部界、集中不等式、...,機率統計學 Probability-and-statistics,統計學,這本書涵蓋了高維統計學的核心方法論和理論，包括尾部界、集中不等式、一致律和經驗過程，以及特定...,Data Science
6,IoT and OT Security Handbook: Assess risks ma...,Leverage Defender for IoT for understanding co...,物聯網 IoT,"物聯網IoT,資訊安全",NaN,Error: 1 validation error for Category\nname\n...
7,游戲專業概論（第4版）,《游戲專業概論（第4版）》融入了游戲動漫行業眾多專業人士的項目製作經驗，結合市場需求，從游戲...,遊戲設計 Game-design,"軟體開發,遊戲設計",NaN,Error: 1 validation error for Category\nname\n...
8,趣味微項目，輕松學Python,主要內容 ● 編寫Python命令行程序 ● 處理Python數據結構 ● 使用和控制隨機性...,Python,Python,這本書主要涵蓋Python程式語言的基礎知識和應用，包括編寫命令行程序、處理數據結構、控制隨...,Python
9,網絡安全與攻防入門很輕松（實戰超值版）,《網絡安全與攻防入門很輕松：實戰超值版》在分析用戶進行黑客防禦中迫切需要用到或迫切想要用到的...,Web-crawler 網路爬蟲,"網路爬蟲,網路管理 MIS,資訊安全",網絡安全與攻防入門很輕松（實戰超值版）涵蓋了網絡安全相關的技術和知識，包括網絡防禦、黑客攻擊...,資訊安全


如果你喜歡 Pydantic 這個概念，可以進一步看 jason 針對 openai library 的擴充 https://jxnl.github.io/instructor/
以及他的課程: https://www.wandb.courses/courses/steering-language-models